# Variable Selection

Variable selection is an important technique in modeling with regression algorithms. When the number of input variables exceed the number of observations, a rank deficiency problem would occur.

When we use ordinary least square methods to approximate the conditional expectation of y with input variable x, we first hypothesis a model such as:
$$ Y = \beta X+ \epsilon$$

Here y is the vector of all dependent variables, and X is the matrix of input feature, with each row representing one observation and each column representing the same feature.

We then solve this equation by multiply $X^T$ to both sides:

$$X^Ty = \beta X^TX+ \epsilon X^T$$

We then take the expected values of both sides and solve the equation, since the expected value of error term is 0, we have

$$X^Ty = \beta X^TX+ 0 X^T$$

$$\mathbb{E}(\beta) = (X^TX)^{-1}X^TY$$

When the number of input variables exceed the number of observations, X is a rank deficient matrix, which means that we cannot take the inverse of $X^T$

If we tried to apply the linear regression algorithms to a rank deficient data, the model will use ordinary least square methods that minimize the mean square error.

$$MSE = \frac{1}{n}\sum_{i=1}^{n}(\text{Residual}_i)^2$$

Thus, we need variable selection to filter out features that contribute little information in predicting the dependent variable.

The possible solution to the problem is Regularization. Regularization methods will put a penalty term on the loss function. Such method will put a constraint on the on the sum of the squared weights. Regularization can help construct the sparsity pattern. Progressive regularization methods will reduce some weights to zero. Then we can conclude that weights of non-zero values have stronger impacts on the prediction of dependent variable. The input variables associated with zero weights can then be excluded from the model, thus completing feature selection.

In this project, I will compare the variable selection efficiency of some common regularization methods including Ridge, Lasso, ElasticNet, Square Root Lasso and Smoothly Clipped Absolute Deviation (SCAD). These methods will be tested with simulated data. Grid search algorithms will be applied to find the optimal hyperparameters that reports highest accuracy in predicting y for each algorithm. I will then compare the cross-validated accuracy, and the mean percentage of true zero and non-zero coefficients of the 10 folds to determine the performance of all those regularization methods on variable selection.




In [ ]:
#import packages
import numpy as np
import pandas as pd
from math import ceil
from scipy import linalg
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import make_spd_matrix
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.linalg import toeplitz
from matplotlib import pyplot
from scipy.optimize import minimize
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.base import BaseEstimator, RegressorMixin
import warnings
warnings.filterwarnings('ignore')

# Simulate Data


For the purpose of this project, I will simulate some data with with weights of zeros for algorithms to detect. After creating the weights, I also encoded it into a binary array with 0 reports weights of 0 and 1 denoting non-zero weights.

In [ ]:
n = 200
p = 1200

In [ ]:
#to create beta* we need to concacenate those portions
p1=[1]*7
p2=[0]*25
p3=[0.25]*5
p4=[0]*50
p5=[0.7]*15
p6=[0]*1098
beta_star = np.concatenate((p1,p2,p3,p4,p5,p6))
beta_star_bi = np.copy(beta_star)
beta_star_bi[beta_star_bi !=0]=1
beta_star_bi = beta_star_bi.astype(int)

In [ ]:
#simulate x, the imput variable
x, the input variable

v = []
for i in  range(p):
  v.append(0.8**i)
np.random.seed(123)
sigma = 3.5
mu=[0]*p
X = np.random.multivariate_normal(mu, toeplitz(v), size=n)

y is then created by multiplication of x and beta with random noise generated to test the algorithm function. A good feature selection algorithm should be able to identify the noise and reconstruct the sparsity pattern of beta



In [ ]:
y = np.matmul(X,beta_star) + sigma*np.random.normal(0,1,n)


#Functions for K fold validation and reporting results

I will use a 10 fold validation to report the accuracy of model by taking the mean squared errors. I will also see how many true zero values and non-zero values will be discovered by the algorithm. Since some algorithm will not reduce the coefficient to zero but rather some values close to zero, I set the threshold of weights to be 1e-05. Weights smaller than 1e-05 will taken as zero, and the algorithms will be considered as successfully filtered out that feature.

In [ ]:
def result_CrossVal(x,y,model):
  mse = []
  dist = []
  mse_beta = []
  dist_beta =[]
  recall_1 = []
  recall_0 = []
  kf = KFold(n_splits=10,shuffle=True,random_state=123)
  for idxtrain, idxtest in kf.split(X):
      xtrain = X[idxtrain]
      ytrain = y[idxtrain]
      ytest = y[idxtest]
      xtest = X[idxtest]
      model.fit(X,y)
      yhat=model.predict(X)
      beta_hat = model.coef_
      mse.append(MSE(y,yhat))
      mse_beta.append(MSE(beta_star,beta_hat))
      dist.append(np.linalg.norm(yhat-y))
      dist_beta.append(np.linalg.norm(beta_hat-beta_star))
      beta_bi = np.copy(beta_hat)
      beta_bi[beta_bi >0.00001]=1
      beta_bi[beta_bi<0.00001]=0
      beta_bi[beta_bi==0.00001]=1
      beta_bi = beta_bi.astype(int)
      report= classification_report(beta_star_bi,beta_bi,output_dict = True)
      recall_1.append(report['1']['recall'])
      recall_0.append(report['0']['recall'])


  return np.mean(mse),np.mean(dist),np.mean(recall_1),np.mean(recall_0),np.mean(mse_beta),np.mean(dist_beta)

In [ ]:
def print_statment(X,name):
  print("The mean squared error of true y and predicted y is %s" %(X[0]))
  print("The euclidean distance between true y and predicted y is %s" %(X[1]))
  print("The percentage of true non-zero coefficients  discovered is %s "%(X[2]))
  print("The percentage of true zero coefficients  discovered by %s "%(X[3]))
  print("The mean squared error of true coefficients and predicted coefficients is %s" %(X[4]))
  print("The euclidean distance between true true coefficients and predicted coefficients is %s" %(X[5]))


## Variable Selection with Lasso

Variable selection with Lasso
Lasso is also know as L1 norm. The penalty term of L1 norm is the sum of absolute values of weights. The least square function with added penalty term will be:

$$\frac{1}{n} \sum_{i=1}^{n} (\text{Residual}_i)^2 + \alpha \|\beta\|_1
$$

alpha is the hyperparameter, it specify how much we penalizing the function to minimize. Grid search methods will be applied to estimate the optimal alpha value. We will use algorithm imported from scikit-learn library



In [ ]:
lasreg = Lasso(fit_intercept = False,max_iter = 10000)
params = [{'alpha':np.linspace(0.001,1,num=200)}]
gs = GridSearchCV(estimator=lasreg,cv=10,scoring='r2',param_grid=params)
gs_results = gs.fit(X,y)
alpha_lasso = gs_results.best_params_.get('alpha')
model = Lasso(alpha = alpha_lasso,fit_intercept = False, max_iter = 10000)
A=result_CrossVal(X,y,model)

In [ ]:
print_statment(A,"Lasso")

The mean squared error of true y and predicted y is 14.95926466392552
The euclidean distance between true y and predicted y is 2600.4266905242175
The percentage of true non-zero coefficients  discovered is 0.7037037037037038 
The percentage of true zero coefficients  discovered by 1.0 
The mean squared error of true coefficients and predicted coefficients is 0.007456596759708996
The euclidean distance between true true coefficients and predicted coefficients is 2.991306756528122


The lasso algorithm successfully found out all the zeros in the weights. However, it might have taken a rather radical step that some non-zero weights were also reduced to zero.

## Variable Selection with Ridge

Ridge is also know as L2 norm. The penalty term of L2 norm is the sum of square root of weights. The least square function with added penalty term will be:

$$\frac{1}{n} \sum_{i=1}^{n} (\text{Residual}_i)^2 + \alpha \sum_{j=1}^{p} \beta_j^2$$


alpha is the hyperparameter, it specify how much we penalizing the function to minimize. Grid search methods will be applied to estimate the optimal alpha value. I will use algorithm imported from scikit-learn library

In [ ]:
Ridgereg = Ridge(fit_intercept = False,max_iter = 10000)
params = [{'alpha':np.linspace(0.1,0.001,num=100)}]
gs = GridSearchCV(estimator=Ridgereg,cv=10,scoring='r2',param_grid=params)
gs_results = gs.fit(X,y)
alpha_R = gs_results.best_params_.get('alpha')


In [ ]:
Ridreg = Ridge(alpha = alpha_R,fit_intercept = False,max_iter = 10000)
R_result=result_CrossVal(X,y,Ridreg)
print_statment(R_result,"Ridge")

The mean squared error of true y and predicted y is 9.05992338232221e-11
The euclidean distance between true y and predicted y is 0.0001346099801821708
The percentage of true non-zero coefficients  discovered is 1.0 
The percentage of true zero coefficients  discovered by 0.5251491901108271 
The mean squared error of true coefficients and predicted coefficients is 0.007514031264213061
The euclidean distance between true true coefficients and predicted coefficients is 3.002804941559753


Ridge is not as efficient as Lasso on feature selection, it only discovers 52.5% of the zeros in weights. However, ridge will avoid making mistakes in recognizing non-zero values into zeros.

## Variable Selection with ElasticNet

ElasticNet combines L1 and L2 regularizations. The penalty term combines the absolute value of weights and the squared terms of weights, so the function with mean squared error to be minimize is:

$$\frac{1}{n} \sum_{i=1}^{n} (\text{Residual}_i)^2 + \lambda \sum_{j=1}^{p} |\beta_j| + (1 - \lambda) \cdot \sum_{j=1}^{p} \beta_j^2$$


In [ ]:
ENreg = ElasticNet(fit_intercept = False,max_iter = 10000)
params = [{'alpha':np.linspace(0.01,0.1,num=10),'l1_ratio':np.linspace(0.1,1,num=10)}]
gs = GridSearchCV(estimator=ENreg,cv=10,scoring='r2',param_grid=params)
gs_results = gs.fit(X,y)
alpha_EN = gs_results.best_params_.get('alpha')
l1_EN = gs_results.best_params_.get('l1_ratio')
model = ElasticNet(alpha = alpha_EN,l1_ratio = l1_EN, fit_intercept = False, max_iter = 10000)
A=result_CrossVal(x,y,model)

In [ ]:
print_statment(A,"ElasticNet")

The mean squared error of true y and predicted y is 0.16777371964906712
The euclidean distance between true y and predicted y is 2882.7721156410175
The percentage of true non-zero coefficients  discovered is 1.0 
The percentage of true zero coefficients  discovered by 0.8312020460358056 
The mean squared error of true coefficients and predicted coefficients is 0.004902890785113761
The euclidean distance between true true coefficients and predicted coefficients is 2.4255863089439873


Elastic performs better on defining the sparsity pattern of weights, as it discovers 83% of true zero coefficient without reduce any non-zero coefficients to zero. Although it did not performs as progressive in feature selection as Lasso regression, note that the accuracy of predicted y denoted by mean square error is lower than that of Lasso.

## Variable Selection with Square Root Lasso

Square root lasso is modification of Lasso regularization, specially developed to estimate high-dimensional sparse linear regression. The objective function we want to minimize with penalty is

$$\sqrt{\frac{1}{n} \sum_{i=1}^{n} (\text{Residual}_i)^2} + \alpha \sum_{i=1}^{n} |\beta_i|$$




In [ ]:
class SQRTLasso(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=0.01):
        self.alpha = alpha

    def fit(self, x, y):
        alpha=self.alpha

        def f_obj(x,y,beta,alpha):
          n =len(x)
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = np.sqrt(1/n*np.sum((y-x.dot(beta))**2)) + alpha*np.sum(np.abs(beta))
          return output

        def f_grad(x,y,beta,alpha):
          n=x.shape[0]
          p=x.shape[1]
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          output = (-1/np.sqrt(n))*np.transpose(x).dot(y-x.dot(beta))/np.sqrt(np.sum((y-x.dot(beta))**2))+alpha*np.sign(beta)
          return output.flatten()

        def objective(beta):
          return(f_obj(x,y,beta,alpha))

        def gradient(beta):
          return(f_grad(x,y,beta,alpha))

        beta0 = np.ones((x.shape[1],1))
        output = minimize(objective, beta0, method='L-BFGS-B', jac=gradient,options={'gtol': 1e-8, 'maxiter': 50000,'maxls': 25,'disp': True})
        beta = output.x
        self.coef_ = beta
    def coef_(self):
      return self.coef_

    def predict(self, x):
        return x.dot(self.coef_)



In [ ]:
np.random.seed(123)
x = np.random.multivariate_normal(mu, toeplitz(v), size=n)
y = np.matmul(x,beta_star).reshape(-1,1) + sigma*np.random.normal(0,1,size=(n,1))


In [ ]:
sqrtLasso = SQRTLasso()
params = [{'alpha':np.linspace(0.1,1,num=10)}]
gs = GridSearchCV(estimator=sqrtLasso,cv=10,scoring='r2',param_grid=params)
gs_results = gs.fit(X,y)
alpha_sqrtLasso  = gs_results.best_params_.get('alpha')

In [ ]:
model = SQRTLasso(alpha = 0.2)
A=result_CrossVal(X,y,model)
print_statment(A,"SQRTLasso")

The mean squared error of true y and predicted y is 13.976022056637756
The euclidean distance between true y and predicted y is 2636.310778214968
The percentage of true non-zero coefficients  discovered is 0.962962962962963 
The percentage of true zero coefficients  discovered by 0.988917306052856 
The mean squared error of true coefficients and predicted coefficients is 0.0012223481215289157
The euclidean distance between true true coefficients and predicted coefficients is 1.211122514791422


Square root lasso yields both high percentage of correctly classified zero and non-zero weights. The percentage of true non-zero weights discovered is 96.3% and that of true zero weights discovered is 98.9%. It outperforms Lasso on correctly locating the zero coefficients without falsely reduced non-zero betas to zero.



## Variable Selection with SCAD

The smoothly clipped absolute deviation is a methods developed to address the problem of introducing bias by LASSO algorithm. The penalty term $p(\beta)$ for SCAD is defined by its derivative

$$p'_\lambda(\beta) = \lambda \left\{ I(\beta \leq \lambda) + \frac{\alpha \lambda - \beta}{(\alpha - 1)\lambda} + I(\beta > \lambda) \right\}
$$

$\alpha$ and $\lambda$ are hyperparameters that controls the penalty. We can see from this algorithm that it will not penalize larger value of $\beta$. For small values of $\beta$, it will be penalized by $\lambda|\beta|$. For $\beta$ between $\lambda$ and $\alpha\lambda$, the objective function will be penalized by $\frac{\alpha \lambda |\beta| - \beta^2 - \lambda^2}{\alpha - 1}$


In [ ]:
class SCAD(BaseEstimator, RegressorMixin):
  def __init__(self, a=1,lam=1):
    self.a = a
    self.lam = lam

  def fit(self, x, y):
      a=self.a
      lam = self.lam


      n = x.shape[0]
      p = x.shape[1]
      def scad_penalty(beta_hat, lambda_val, a_val):
          is_linear = (np.abs(beta_hat) <= lambda_val)
          is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
          is_constant = (a_val * lambda_val) < np.abs(beta_hat)

          linear_part = lambda_val * np.abs(beta_hat) * is_linear
          quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
          constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
          return linear_part + quadratic_part + constant_part

      def scad_derivative(beta_hat, lambda_val, a_val):
          return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))


      def scad(beta,x,y):
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          n = len(y)
          return 1/n*np.sum((y-x.dot(beta))**2) + np.sum(scad_penalty(beta,lam,a))

      def dscad(beta,x,y):
          beta = beta.flatten()
          beta = beta.reshape(-1,1)
          n = len(y)
          return np.array(-2/n*np.transpose(x).dot(y-x.dot(beta))+scad_derivative(beta,lam,a)).flatten()

      def objective(beta):
          return scad(beta,x,y)

      def gradient(beta):
          return dscad(beta,x,y)

      b0 = np.ones((p,1))
      output = minimize(objective, b0, method='L-BFGS-B', jac=gradient,options={'gtol': 1e-8, 'maxiter': 1e7,'maxls': 25,'disp': True})
      beta = output.x
      self.coef_ = beta

  def coef_(self):
    return self.coef_

  def predict(self, x):
    return x.dot(self.coef_)


In [ ]:
np.random.seed(123)
x = np.random.multivariate_normal(mu, toeplitz(v), size=n)
y = np.matmul(x,beta_star).reshape(-1,1) + sigma*np.random.normal(0,1,size=(n,1))

In [ ]:
SCADreg = SCAD()
params = [{'a':np.linspace(10,1,num=10),'lam':np.linspace(1,0.1,num=10)}]
gs = GridSearchCV(estimator=SCADreg,cv=10,scoring='neg_mean_squared_error',param_grid=params)
gs_results = gs.fit(X,y)
a_SCAD= gs_results.best_params_.get('a')
lam_SCAD = gs_results.best_params_.get('lam')

In [ ]:
model = SCAD(a=a_SCAD,lam=lam_SCAD)
A=result_CrossVal(X,y,model)
print_statment(A,"SCAD")

The mean squared error of true y and predicted y is 3.604251976224318
The euclidean distance between true y and predicted y is 2919.161486375872
The percentage of true non-zero coefficients  discovered is 1.0 
The percentage of true zero coefficients  discovered by 0.6683716965046889 
The mean squared error of true coefficients and predicted coefficients is 0.014587625433907228
The euclidean distance between true true coefficients and predicted coefficients is 4.183915692349532


The SCAD function didn't performed well on feature selection. The percentage of true zero weights it discovered, 66.8% are more than those of Ridge, but less than others methods.



#Conclusion

Among the regularization methods we experimented with simulated dataset, Square Root Lasso performed the best with feature selection and constructing the sparsity pattern of weights.

Reference:

 https://andrewcharlesjones.github.io/journal/scad.html Belloni, A., Chernozhukov, V., & Wang, L. (2011). Square-root lasso: Pivotal recovery of sparse signals via conic programming. SSRN Electronic Journal. https://doi.org/10.2139/ssrn.1910753